In [ ]:
!pip install transformers==3.5.0
!pip install nltk
!pip install biobert-pytorch
!pip install wget
!pip install cookiecutter

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD" -O biobert_weights && rm -rf /tmp/cookies.txt

!tar -xzf biobert_weights

!transformers-cli convert --model_type bert --tf_checkpoint biobert_v1.1_pubmed/model.ckpt-1000000 --config biobert_v1.1_pubmed/bert_config.json --pytorch_dump_output biobert_v1.1_pubmed/pytorch_model.bin

!mv biobert_v1.1_pubmed/bert_config.json biobert_v1.1_pubmed/config.json

--2020-12-15 10:25:12--  https://docs.google.com/uc?export=download&confirm=CHdY&id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD
Resolving docs.google.com (docs.google.com)... 74.125.142.113, 74.125.142.101, 74.125.142.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-9k-docs.googleusercontent.com/docs/securesc/prbajjq90jv86var83iu444vhsc8dgeu/f2p3h1hcf5sh15cf3u5r3ortd0mplq7d/1608027900000/13799006341648886493/00568481803443554857Z/1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD?e=download [following]
--2020-12-15 10:25:12--  https://doc-14-9k-docs.googleusercontent.com/docs/securesc/prbajjq90jv86var83iu444vhsc8dgeu/f2p3h1hcf5sh15cf3u5r3ortd0mplq7d/1608027900000/13799006341648886493/00568481803443554857Z/1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD?e=download
Resolving doc-14-9k-docs.googleusercontent.com (doc-14-9k-docs.googleusercontent.com)... 74.125.142.132, 2607:f8b0:400e:c08::84
Connecting

In [ ]:
import os
import io
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertForPreTraining
import nltk
from sklearn.model_selection import train_test_split
import re
from transformers import BertTokenizer, BertModel
from transformers import BertForTokenClassification, AdamW
from transformers import tokenization_utils_base
import torch
import torch.nn.functional as F
import torch.nn as nn
import pickle
from torch.utils.data import TensorDataset, DataLoader
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
TEXT_PATH="/content/drive/My Drive/MeasEval-main/data/train/test/text"
TSV_PATH="/content/drive/My Drive/MeasEval-main/data/train/dev/tsv"
MEASURED_ENTITY_MODEL_PATH="/content/drive/My Drive/Model/biobert_measured_entity.pt"
MEASURED_QUANTITY_MODEL_PATH="/content/drive/My Drive/Model/quantity_mixedloss_t55_sh.pt"
MEASURED_PROPERTY_MODEL_PATH="/content/drive/My Drive/Model/biobert_measured_prop.pt"
QUALIFIER_PATH="/content/drive/My Drive/Model/qualifier_20f1_t995.pt"
COUNT_MEASUREMENT_MODEL_PATH="/content/drive/My Drive/Model/count_t95_f70_ep20_sirdata.pt"
INFORMATION_DETECTION_PATH="/content/drive/My Drive/Model/info_60f1_t325.pt"
UNIT_DETECTION_PATH="/content/drive/My Drive/Model/units_t75_96f1.pt"
VOCAB_PATH="/content/drive/My Drive/Model/vocab.Field"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_BioBert = BertModel.from_pretrained('/content/biobert_v1.1_pubmed')
tokenizer_BioBert = BertTokenizer(vocab_file='biobert_v1.1_pubmed/vocab.txt', do_lower_case=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
files=os.listdir(TEXT_PATH)

In [ ]:
test_text=[]
id=[]
j=1
for i in files:

  pathtxt=os.path.join(TEXT_PATH,i)
  
  with open(pathtxt, 'r') as f:
    text=f.read()
  
  sent_text = nltk.sent_tokenize(text)
  for l in sent_text:
    test_text.append(l)
    id.append(j)
  j+=1

In [ ]:
test_text

['(F) Adult hRPE cultures display expression of markers typical of native RPE in their polarized localization.',
 'DAPI is cyan, whereas all other immunofluorescence is gold.',
 'Claudin 19, ezrin, ZO1, and MCT1 are preferentially located on the apical side.',
 'RPE65 and CRALBP are cytoplasmic.',
 'The scale bars represent 10 μm.',
 'GFP-expressing H9, BBHX8, and A1ATD-1 cells were generated by stable transfection using lipofectamine 2000 (Invitrogen) as described previously (Vallier et al., 2001).',
 'GFP-positive cells were differentiated into foregut cells and then dissociated into single cells.',
 'An individually isolated GFP cell was then transferred into a well containing non-GFP-positive hFSCs.',
 'Wells were visually inspected 12 hr after plating, and wells containing a single GFP-positive hFSC were selected for clonal expansion.',
 'For group analyses, the following procedures were applied.',
 '(1) T1-weighted sMRIs from each subject (Fig.',
 '1A) were registered to MNI spac

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      self.bilstm = nn.LSTM(embed_dim, hidden_dim,  bidirectional=True, batch_first=True)
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*hidden_dim,out_dim)
      self.sigmoid = nn.Sigmoid()

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      x = self.bert(sent_id, attention_mask=mask)
      embed = x[0]
      cls_hs = x[1]
      x,_ = self.bilstm(embed)
      x = self.dropout(x)
      x = self.fc1(x)
      x = self.sigmoid(x)

      return x
      
bert_model = BERT_Arch(model_BioBert, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)


Measured Property

In [ ]:
bert_model = torch.load(MEASURED_PROPERTY_MODEL_PATH)
Measured_Property=[]

for s in test_text:
  tokenized_text = tokenizer_BioBert.tokenize(s)
  test_inputs = tokenizer_BioBert(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
  bert_model.eval()
  temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.30
  Measured_Property.append(temp[0])

In [ ]:
Property=[]
for i in range(len(test_text)):
  tokenized_text = tokenizer_BioBert.tokenize(test_text[i])
  txt=[]
  label=[]
  # print(test_text[i])
  combine=""
  for j in range(len(tokenized_text)):
    if(Measured_Property[i][j+1] and Measured_Property[i][j] and j!=0):
      label.append(tokenized_text[j])
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
          combine=combine+tokenized_text[j][2:]
      
      else:
        if(len(combine)==0):
          combine=tokenized_text[j]
          continue
        
        if(tokenized_text[j] in ("!", "," ,"\'" ,";" ,"\"", ".", "-" ,"?","+","-","*","/","%")):
          combine=combine+tokenized_text[j]
          continue
        if(tokenized_text[j] in ("(",")") or combine in ("(",")")):
          continue
  
        start=[m.start() for m in re.finditer(combine,test_text[i])]
        end=[m.start() for m in re.finditer(tokenized_text[j],test_text[i])]
        flag=-1
        
        for s in start:
          for e in end:
            if(flag!=-1):
              break
            
            
            if(e-s-len(combine)+1==1):
              flag=1
            elif(e-s-len(combine)+1==2):
              flag=2
        
        if(flag==2):
          combine=combine+" "+tokenized_text[j]
        elif(flag==1):
          combine=combine+tokenized_text[j]
          
        else:
          txt.append(combine)
          combine=""

    elif(Measured_Property[i][j+1]):
      label.append(tokenized_text[j])
      
      if(len(combine)!=0):
        txt.append(combine)
      
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
        tokenized_text[j]=tokenized_text[j][2:]
      
      combine=tokenized_text[j]
    else:
      if(len(combine)!=0):
        txt.append(combine)
      
      combine=""
  if(len(combine)!=0):
        txt.append(combine)
  
  print(test_text[i])
  print(txt)
  Property.append(txt)


(F) Adult hRPE cultures display expression of markers typical of native RPE in their polarized localization.
[]
DAPI is cyan, whereas all other immunofluorescence is gold.
[]
Claudin 19, ezrin, ZO1, and MCT1 are preferentially located on the apical side.
[]
RPE65 and CRALBP are cytoplasmic.
[]
The scale bars represent 10 μm.
['scale bars']
GFP-expressing H9, BBHX8, and A1ATD-1 cells were generated by stable transfection using lipofectamine 2000 (Invitrogen) as described previously (Vallier et al., 2001).
[]
GFP-positive cells were differentiated into foregut cells and then dissociated into single cells.
[]
An individually isolated GFP cell was then transferred into a well containing non-GFP-positive hFSCs.
[]
Wells were visually inspected 12 hr after plating, and wells containing a single GFP-positive hFSC were selected for clonal expansion.
[]
For group analyses, the following procedures were applied.
[]
(1) T1-weighted sMRIs from each subject (Fig.
[]
1A) were registered to MNI space

Measured Entity

In [ ]:
bert_model = torch.load(MEASURED_ENTITY_MODEL_PATH)
Measured_Entity=[]
for s in test_text:
  tokenized_text = tokenizer_BioBert.tokenize(s)
  test_inputs = tokenizer_BioBert(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
  bert_model.eval()
  temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.30
  Measured_Entity.append(temp[0])

In [ ]:
Entity=[]
for i in range(len(test_text)):
  tokenized_text = tokenizer_BioBert.tokenize(test_text[i])
  txt=[]
  label=[]
  
  combine=""
  for j in range(len(tokenized_text)):
    if(Measured_Entity[i][j+1] and Measured_Entity[i][j] and j!=0):
      label.append(tokenized_text[j])
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
          combine=combine+tokenized_text[j][2:]
      
      else:
        if(len(combine)==0):
          combine=tokenized_text[j]
          continue
        
        if(tokenized_text[j] in ("!", "," ,"\'" ,";" ,"\"", ".", "-" ,"?","+","-","*","/","%")):
          combine=combine+tokenized_text[j]
          continue
        if(tokenized_text[j] in ("(",")") or combine in ("(",")")):
          continue
  
        start=[m.start() for m in re.finditer(combine,test_text[i])]
        end=[m.start() for m in re.finditer(tokenized_text[j],test_text[i])]
        flag=-1
        
        for s in start:
          for e in end:
            if(flag!=-1):
              break
            
            
            if(e-s-len(combine)+1==1):
              flag=1
            elif(e-s-len(combine)+1==2):
              flag=2
        
        if(flag==2):
          combine=combine+" "+tokenized_text[j]
        elif(flag==1):
          combine=combine+tokenized_text[j]
          
        else:
          txt.append(combine)
          combine=""

    elif(Measured_Entity[i][j+1]):
      label.append(tokenized_text[j])
      
      if(len(combine)!=0):
        txt.append(combine)
      
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
        tokenized_text[j]=tokenized_text[j][2:]
      
      combine=tokenized_text[j]
    else:
      if(len(combine)!=0):
        txt.append(combine)
      
      combine=""
  if(len(combine)!=0):
        txt.append(combine)
  
  print(test_text[i])
  print(txt)  
  Entity.append(txt)
        


(F) Adult hRPE cultures display expression of markers typical of native RPE in their polarized localization.
[]
DAPI is cyan, whereas all other immunofluorescence is gold.
[]
Claudin 19, ezrin, ZO1, and MCT1 are preferentially located on the apical side.
[]
RPE65 and CRALBP are cytoplasmic.
[]
The scale bars represent 10 μm.
['scale bars', '.']
GFP-expressing H9, BBHX8, and A1ATD-1 cells were generated by stable transfection using lipofectamine 2000 (Invitrogen) as described previously (Vallier et al., 2001).
['GFP-', ',']
GFP-positive cells were differentiated into foregut cells and then dissociated into single cells.
[]
An individually isolated GFP cell was then transferred into a well containing non-GFP-positive hFSCs.
['GFP cell']
Wells were visually inspected 12 hr after plating, and wells containing a single GFP-positive hFSC were selected for clonal expansion.
['Wells']
For group analyses, the following procedures were applied.
[]
(1) T1-weighted sMRIs from each subject (Fig.
['

Quantity

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)
bert_model = torch.load(MEASURED_QUANTITY_MODEL_PATH)
Measured_Quantity=[]
for s in test_text:
  tokenized_text = tokenizer.tokenize(s)
  test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
  bert_model.eval()
  temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.30
  Measured_Quantity.append(temp[0])
  

In [ ]:
Quantity=[]
for i in range(len(test_text)):
  tokenized_text = tokenizer.tokenize(test_text[i])
  txt=[]
  label=[]
  
  combine=""
  for j in range(len(tokenized_text)):
    if(Measured_Quantity[i][j+1] and Measured_Quantity[i][j] and j!=0):
      label.append(tokenized_text[j])
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
          combine=combine+tokenized_text[j][2:]
      
      else:
        if(len(combine)==0):
          combine=tokenized_text[j]
          continue
        
        if(tokenized_text[j] in ("!", "," ,"\'" ,";" ,"\"", ".", "-" ,"?","+","-","*","/","%")):
          combine=combine+tokenized_text[j]
          continue
        if(tokenized_text[j] in ("(",")") or combine in ("(",")")):
          print(combine)
          continue
        start=[m.start() for m in re.finditer(combine,test_text[i])]
        end=[m.start() for m in re.finditer(tokenized_text[j],test_text[i])]
        flag=-1
        
        for s in start:
          for e in end:
            if(flag!=-1):
              break
            
            
            if(e-s-len(combine)+1==1):
              flag=1
            elif(e-s-len(combine)+1==2):
              flag=2
        
        if(flag==2):
          combine=combine+" "+tokenized_text[j]
        elif(flag==1):
          combine=combine+tokenized_text[j]
          
        else:
          txt.append(combine)
          combine=""

    elif(Measured_Quantity[i][j+1]):
      label.append(tokenized_text[j])
      
      if(len(combine)!=0):
        txt.append(combine)
      
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
        tokenized_text[j]=tokenized_text[j][2:]
      
      combine=tokenized_text[j]
    else:
      if(len(combine)!=0):
        txt.append(combine)
      
      combine=""
        
  if(len(combine)!=0):
        txt.append(combine)
  
               
  print(test_text[i])
  print(txt)
  Quantity.append(txt)
  
  

(F) Adult hRPE cultures display expression of markers typical of native RPE in their polarized localization.
[]
DAPI is cyan, whereas all other immunofluorescence is gold.
[]
Claudin 19, ezrin, ZO1, and MCT1 are preferentially located on the apical side.
[]
RPE65 and CRALBP are cytoplasmic.
[]
The scale bars represent 10 μm.
['10 μm.']
GFP-expressing H9, BBHX8, and A1ATD-1 cells were generated by stable transfection using lipofectamine 2000 (Invitrogen) as described previously (Vallier et al., 2001).
[]
GFP-positive cells were differentiated into foregut cells and then dissociated into single cells.
[]
An individually isolated GFP cell was then transferred into a well containing non-GFP-positive hFSCs.
[]
Wells were visually inspected 12 hr after plating, and wells containing a single GFP-positive hFSC were selected for clonal expansion.
['12 hr']
For group analyses, the following procedures were applied.
[]
(1) T1-weighted sMRIs from each subject (Fig.
[]
1A) were registered to MNI sp

Qualifier

In [ ]:
bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)
bert_model = torch.load(QUALIFIER_PATH)
Qual=[]
for s in test_text:
  tokenized_text = tokenizer.tokenize(s)
  test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
  bert_model.eval()
  temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.995
  
  Qual.append(temp[0])

In [ ]:
Qualifier=[]
for i in range(len(test_text)):
  tokenized_text = tokenizer.tokenize(test_text[i])
  txt=[]
  label=[]
  
  combine=""
  for j in range(len(tokenized_text)):
    if(Qual[i][j+1] and Qual[i][j] and j!=0):
      label.append(tokenized_text[j])
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
          combine=combine+tokenized_text[j][2:]
      
      else:
        if(len(combine)==0):
          combine=tokenized_text[j]
          continue
        
        if(tokenized_text[j] in ("!", "," ,"\'" ,";" ,"\"", ".", "-" ,"?","+","-","*","/","%")):
          combine=combine+tokenized_text[j]
          continue
        if(tokenized_text[j] in ("(",")") or combine in ("(",")")):
          print(combine)
          continue
        start=[m.start() for m in re.finditer(combine,test_text[i])]
        end=[m.start() for m in re.finditer(tokenized_text[j],test_text[i])]
        flag=-1
        
        for s in start:
          for e in end:
            if(flag!=-1):
              break
            
            
            if(e-s-len(combine)+1==1):
              flag=1
            elif(e-s-len(combine)+1==2):
              flag=2
        
        if(flag==2):
          combine=combine+" "+tokenized_text[j]
        elif(flag==1):
          combine=combine+tokenized_text[j]
          
        else:
          txt.append(combine)
          combine=""

    elif(Qual[i][j+1]):
      label.append(tokenized_text[j])
      
      if(len(combine)!=0):
        txt.append(combine)
      
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
        tokenized_text[j]=tokenized_text[j][2:]
      
      combine=tokenized_text[j]
    else:
      if(len(combine)!=0):
        txt.append(combine)
      
      combine=""
        
  if(len(combine)!=0):
        txt.append(combine)
  
               
  print(test_text[i])
  print(txt)
  Quantity.append(txt)
  
  

(F) Adult hRPE cultures display expression of markers typical of native RPE in their polarized localization.
[]
DAPI is cyan, whereas all other immunofluorescence is gold.
[]
Claudin 19, ezrin, ZO1, and MCT1 are preferentially located on the apical side.
[]
RPE65 and CRALBP are cytoplasmic.
[]
The scale bars represent 10 μm.
[]
GFP-expressing H9, BBHX8, and A1ATD-1 cells were generated by stable transfection using lipofectamine 2000 (Invitrogen) as described previously (Vallier et al., 2001).
[]
GFP-positive cells were differentiated into foregut cells and then dissociated into single cells.
[]
An individually isolated GFP cell was then transferred into a well containing non-GFP-positive hFSCs.
[]
Wells were visually inspected 12 hr after plating, and wells containing a single GFP-positive hFSC were selected for clonal expansion.
[]
For group analyses, the following procedures were applied.
[]
(1) T1-weighted sMRIs from each subject (Fig.
[]
1A) were registered to MNI space (Montreal N

Count and Measurement

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      self.bilstm = nn.LSTM(embed_dim, hidden_dim, n_layers,dropout=drop_prob,  bidirectional=True, batch_first=True)
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*hidden_dim,256)
      self.r1=nn.Tanh()
      self.fc2 = nn.Linear(256,out_dim)
      self.sf = nn.Sigmoid()

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      embed, cls_hs = self.bert(sent_id, attention_mask=mask)
      x,_ = self.bilstm(embed)
      x = self.dropout(x)
      x = self.fc1(x)
      x = self.r1(x)
      x = self.fc2(x)
      x = self.sf(x)

      return x

bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)


In [ ]:
bert_model = torch.load(COUNT_MEASUREMENT_MODEL_PATH)
count_measurement=[]
for s in test_text:
  tokenized_text = tokenizer.tokenize(s)
  test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
  bert_model.eval()
  temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.95
  count_measurement.append(temp[0])

In [ ]:
for i in range(len(test_text)):
  tokenized_text = tokenizer.tokenize(test_text[i])
  txt=[]
  print(test_text[i])
  for j in range(len(tokenized_text)):
    if(count_measurement[i][j+1]):
      txt.append(tokenized_text[j])   
  print(txt)

Extra Information

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      self.n_layers = n_layers
      self.bilstm = nn.LSTM(embed_dim, hidden_dim,  bidirectional=True, batch_first=True)
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*hidden_dim,out_dim)
      self.sigmoid = nn.Sigmoid()

    #define the forward pass
    def forward(self, sent_id, mask):

      
      # embed,cls_hs = self.bert(sent_id, attention_mask=mask)
      
      x = self.bert(sent_id, attention_mask=mask)
      embed=x[0]
      cls_hs=x[1]
      
      x,_ = self.bilstm(embed)
      
      x = self.dropout(x)
      x = self.fc1(x)
      x = self.sigmoid(x)

      return x

bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)

In [ ]:
bert_model = torch.load(INFORMATION_DETECTION_PATH)
info=[]
for s in test_text:
  tokenized_text = tokenizer.tokenize(s)
  test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
  bert_model.eval()
  temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.3
  info.append(temp[0])

In [ ]:
idx2labels = {0:'HasTolerance',
 1:'IsApproximate',
 2:'IsCount',
 3:'IsList',
 4:'IsMean',
 5:'IsMeanHasSD',
 6:'IsMeanHasTolerance',
 7:'IsMeanIsRange',
 8:'IsMedian',
 9:'IsRange',
 10:'IsRangeHasTolerance'}

In [ ]:
for i in range(len(test_text)):
  tokenized_text = tokenizer.tokenize(test_text[i])
  txt=[]
  print(test_text[i])
  
  for j in range(len(tokenized_text)):
    for k in range(11):
      if(info[i][j+1][k]):
        txt.append(tokenized_text[j]+" "+idx2labels[k])   
  print(txt)

(F) Adult hRPE cultures display expression of markers typical of native RPE in their polarized localization.
[]
DAPI is cyan, whereas all other immunofluorescence is gold.
[]
Claudin 19, ezrin, ZO1, and MCT1 are preferentially located on the apical side.
[]
RPE65 and CRALBP are cytoplasmic.
[]
The scale bars represent 10 μm.
[]
GFP-expressing H9, BBHX8, and A1ATD-1 cells were generated by stable transfection using lipofectamine 2000 (Invitrogen) as described previously (Vallier et al., 2001).
[]
GFP-positive cells were differentiated into foregut cells and then dissociated into single cells.
[]
An individually isolated GFP cell was then transferred into a well containing non-GFP-positive hFSCs.
[]
Wells were visually inspected 12 hr after plating, and wells containing a single GFP-positive hFSC were selected for clonal expansion.
['hr IsRange']
For group analyses, the following procedures were applied.
[]
(1) T1-weighted sMRIs from each subject (Fig.
[]
1A) were registered to MNI space

Unit Detection

In [ ]:
file = open(VOCAB_PATH,'rb')
TEXT = pickle.load(file)
vocab_size=len(TEXT.vocab)
class bilstm(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.output_size = output_size
        
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                         batch_first=True, bidirectional=True)
        
        self.dropout = nn.Dropout(0)
        self.fc = nn.Linear(hidden_dim*2,output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        embeds = self.embedding(x)
        lstm, (hn,cn)= self.lstm(embeds)
        
        output = self.dropout(lstm)
        output = self.fc(output)
        output = self.sigmoid(output) 
        return output
device = "cpu"
model = bilstm(vocab_size, 1, 100, 32, 1)
model = model.to(device)

In [ ]:
model = torch.load(UNIT_DETECTION_PATH)

In [ ]:
UNITS=[]

for i in Quantity:
  unit=[]
  for j in i:
    X_train=[]
    X_train.append(TEXT.process(j)[0].numpy())
    
    X_train=np.array(X_train)

    
    X_train=torch.from_numpy(X_train)
    X_train=X_train[:,:,0]

    
    row=X_train.shape[1]
    
    
    X_train=F.pad(input=X_train, pad=(0, 64-row, 0, 0), mode='constant', value=1)
    
    model.zero_grad()
    model.eval()
    y_pred = model(X_train.to(device))
    
    np_out = y_pred.cpu().data.numpy()
    
    np_out=np_out>=0.75
    un=""
    for k in range(len(np_out[0])):
      if(np_out[0][k]):
        un=un+j[k]
    if(len(un)==0):
      print("Quantity-"+j,"Unit-None")
    else:
      print("Quantity-"+j,"Unit-"+un)
    unit.append(un)

  UNITS.append(unit)

Quantity-10 μm. Unit-μm
Quantity-12 hr Unit-hr
Quantity-96 Unit-None
Quantity-5 mm per side Unit-mmper side
Quantity-two Unit-None
Quantity-after 6 months Unit-months
Quantity-more than 50,000 Unit-None
Quantity-1/10 that Unit-h
Quantity-2 months, Unit-months
Quantity-3.5–4 months Unit-months
Quantity-1/100 Unit-None
Quantity-4 weeks Unit-weeks
Quantity-6 Unit-None
Quantity-125 μm Unit-μm
Quantity-25 μm Unit-μm
Quantity-100% Unit-%
Quantity-3–4 Unit-None
Quantity-5–55 Unit-None
Quantity-30–130 ms Unit-ms
Quantity-4d Unit-None
Quantity-95.81% Unit-%
Quantity-94.38% Unit-%
Quantity-96.24% Unit-%
Quantity-93.17% Unit-%
Quantity-1.16, Unit-None
Quantity-0.25 Unit-None
Quantity-1.31, Unit-None
Quantity-50 μm. Unit-μm
Quantity-3 months Unit-months
Quantity-36 Unit-None
Quantity-26 Unit-None
Quantity-one Unit-None
Quantity-more than 20 passages. Unit-passages
Quantity-100% Unit-%
Quantity-greater than 70%. Unit-%
Quantity-25 days. Unit-days
Quantity-4 weeks Unit-weeks
Quantity-6 weeks Unit-ee

Relation Extraction

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """
    # print(len(sentence))
    # print(len(text_labels))
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)
    # print(len(tokenized_sentence))
    # print(len(labels))
    return tokenized_sentence, labels

In [ ]:
lis_en = []
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
for i in range(len(test_text)):
  tag={}
  embed={} 
  for j in range(len(Quantity[i])):
    L=test_text[i].lower()
    if L.find(Quantity[i][j]):
      start=[m.start() for m in re.finditer(Quantity[i][j],L)]
      if(len(start)>0):
        start_q = start[0]
        end_q = len(Quantity[i][j]) + start_q - 1
        tag["Q-"+str(j+1)]=[start_q,end_q,Quantity[i][j]]
        embed["Q-"+str(j+1)]=np.zeros(768) 
  for j in range(len(Property[i])):
    test_text[i]=test_text[i].lower()
    if test_text[i].find(Property[i][j]):
      
      if(Property[i][j] in ("!", "," ,"\'" ,";" ,"\"", ".", "-" ,"?",")","+","-","*","/","%")):
        continue
      start=[m.start() for m in re.finditer(Property[i][j],test_text[i])]
      
      if(len(start)>0):
        start_q = start[0]
        end_q = len(Property[i][j]) + start_q - 1
        tag["P-"+str(j+1)]=[start_q,end_q,Property[i][j]]
        embed["P-"+str(j+1)]=np.zeros(768)
  for j in range(len(Entity[i])):
    
    if test_text[i].find(Entity[i][j]):
      if(Entity[i][j] in ("!", "," ,"\'" ,";" ,'(',"\"", ".", "-" ,"?",")","+","-","*","/","%")):
        continue
      
      start=[m.start() for m in re.finditer(Entity[i][j],test_text[i])]
      # print(start,test_text[i],Entity[i][j])
      if(len(start)>0):
        start_q = start[0]
        end_q = len(Entity[i][j]) + start_q - 1
        tag["E-"+str(j+1)]=[start_q,end_q,Entity[i][j]]
        embed["E-"+str(j+1)]=np.zeros(768)

  
  
  j=test_text[i]
  ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
  entities=[]
  for word in ret:
    temp=[]
    for key,ent_doc in tag.items():
      if(word[1]>=ent_doc[0] and word[2]<=ent_doc[1]):
        temp.append(key)
    entities.append(temp)
  

  sen,label=tokenize_and_preserve_labels(j.split(' '),entities,tokenizer)
  label.insert(0,[])
  label.append([])
  inputs = tokenizer(j, return_tensors="pt")
  outputs = model(**inputs)
  
  last_hidden_states = outputs[0]

  for i in range(len(label)):
    if(len(label[i])==0):
      continue
    else:
      for k in range(len(label[i])):
        embed[label[i][k]]=embed[label[i][k]]+last_hidden_states[0][i].detach().numpy()
  
  data=[]

  for key,val in tag.items():
    if(key[0]=='Q'):
      data.append(["Quantity",embed[key],val[2]])
    elif(key[0]=='P'):
      data.append(["MeasuredProperty",embed[key],val[2]])
    else:
      data.append(["MeasuredEntity",embed[key],val[2]])    
  lis_en.append(data)
  

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
inputA = layers.Input(shape=(768,))
inputB = layers.Input(shape=(768,))
inputC = layers.Input(shape=(1,))
inputD = layers.Input(shape = (1,))

x = layers.Dense(128, activation="relu")(inputA)
x = layers.Dense(64, activation="relu")(x)

y = layers.Dense(128, activation="relu")(inputB)
y = layers.Dense(64, activation="relu")(y)

c = layers.Dense(1, activation="relu")(inputC)

d = layers.Dense(1, activation="relu")(inputD)


com = layers.concatenate([x, y, c, d])
z = layers.Dense(32, activation="relu")(com)
z = layers.Dense(4,  activation='softmax')(z)
model = keras.Model(inputs=[inputA, inputB, inputC, inputD], outputs=z)

In [ ]:
weights = [15.593220338983052, 10.415094339622641, 33.65853658536585, 2.323232323232323]

In [ ]:
from keras import backend as K
def weighted_categorical_crossentropy(weights):

    weights = K.variable(weights)
        
    def loss(y_true, y_pred):

        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [ ]:
model.compile(optimizer='adam',
              loss= weighted_categorical_crossentropy(weights),
              metrics=['accuracy'])

In [ ]:
from keras.models import load_model
model.load_weights('/content/drive/My Drive/Model/rel_wt_new.h5')

In [ ]:
idx2tok = {}
idx2tok["Quantity"] = 1
idx2tok["MeasuredEntity"] = 2
idx2tok["MeasuredProperty"] = 3
idx2tok["Qualifier"] = 4

In [ ]:
tok2idx = {}
tok2idx[0] = "HasProperty"
tok2idx[1] = "HasQuantity"
tok2idx[2] = "Qualifies"
tok2idx[3] = "norelation"

In [ ]:
relation = []
for sen in lis_en:
  ans = []
  for i in range(len(sen)):
    for j in range(i+1, len(sen)):
      x1 = sen[i][1].reshape([1,768])
      x2 = sen[j][1].reshape([1,768])
      x3 = np.array([idx2tok[sen[i][0]]]).reshape([1,1])
      x4 = np.array([idx2tok[sen[j][0]]]).reshape([1,1])
      
      y_pred = model.predict([x1,x2,x3,x4])
      rel_pred = np.argmax(y_pred)
      if x3[0,0] == x4[0,0]:
        continue
      if tok2idx[rel_pred]=="norelation":
        continue
      if x3[0,0]==1 and x4[0,0]==2 and tok2idx[rel_pred]=="HasQuantity":
        ans.append([sen[j][2], sen[i][2], x4[0,0],x3[0,0], tok2idx[rel_pred]])
      if x3[0,0]==2 and x4[0,0]==1 and tok2idx[rel_pred]=="HasQuantity":
        ans.append([sen[i][2], sen[j][2],x3[0,0],x4[0,0], tok2idx[rel_pred]])
      if x3[0,0]==3 and x4[0,0]==2 and tok2idx[rel_pred]=="HasProperty":
        ans.append([sen[j][2], sen[i][2], x4[0,0],x3[0,0], tok2idx[rel_pred]])
      if x3[0,0]==2 and x4[0,0]==3 and tok2idx[rel_pred]=="HasProperty":
        ans.append([sen[i][2], sen[j][2],x3[0,0],x4[0,0], tok2idx[rel_pred]])
      if x3[0,0]==1 and x4[0,0]==4 and tok2idx[rel_pred]=="Qualifies":
        ans.append([sen[j][2], sen[i][2], x4[0,0],x3[0,0], tok2idx[rel_pred]])
      if x3[0,0]==4 and x4[0,0]==1 and tok2idx[rel_pred]=="Qualifies":
        ans.append([sen[i][2], sen[j][2],x3[0,0],x4[0,0], tok2idx[rel_pred]])
      
  
  relation.append(ans)

[[],
 [],
 [],
 [],
 [['scale bars', '10 μm.', 2, 1, 'HasQuantity'],
  ['scale bars', 'scale bars', 2, 3, 'HasProperty']],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [['regional masks', '5 mm per side', 2, 1, 'HasQuantity'],
  ['source grid', '5 mm per side', 2, 1, 'HasQuantity'],
  ['voxels', '5 mm per side', 2, 1, 'HasQuantity'],
  ['side', '5 mm per side', 2, 1, 'HasQuantity'],
  ['regional masks', 'grid with voxels', 2, 3, 'HasProperty'],
  ['source grid', 'grid with voxels', 2, 3, 'HasProperty'],
  ['voxels', 'grid with voxels', 2, 3, 'HasProperty'],
  ['side', 'grid with voxels', 2, 3, 'HasProperty']],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [['double-blind clinical trials', 'after 6 months', 2, 1, 'HasQuantity'],
  ['double-blind clinical trials', 'used', 2, 3, 'HasProperty'],
  ['immunosuppressive', 'used', 2, 3, 'HasProperty'],
  ['double-blind clinical trials', 'withdrawn', 2, 3, 'HasProperty'],
  ['immunosuppressive', 'withdrawn', 2, 3, 'Has